In [ ]:
# 只做一次
# pip install geopandas libpysal linearmodels scipy pandas numpy
from pathlib import Path   # ← 关键：先导入 Path
import numpy as np
import pandas as pd
import geopandas as gpd

from libpysal import weights
from libpysal.weights import Queen, KNN
from scipy.sparse import identity
from scipy.sparse.linalg import inv as spinv

from linearmodels.iv import IV2SLS   # 用 2SLS/IV 估计 δ、ρ、β、θ（带双向固定效应）

In [ ]:
PATH_PANEL = Path(r"D:\Dissertation\dissertation\data 2\data\Without UK and Germany\Final\panel_long_merged_with_general.csv")
PATH_GEO   = Path(r"D:\Dissertation\dissertation\data 2\data\Without UK and Germany\Final\cleaned geo data\NUTS2_2021.geojson")
OUT_DIR    = Path(r"D:\Dissertation\dissertation\data 2\data\Without UK and Germany\Final\moran\SDM_Edit")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# ── 读数据 ─────────────────────────────────────────────────────────────
panel = pd.read_csv(PATH_PANEL)                       # 长表：NUTS_ID, year, ...
nuts  = gpd.read_file(PATH_GEO)[["NUTS_ID","geometry"]]

In [ ]:
panel = panel.rename(columns={"region": "NUTS_ID"})   # 关键一行
panel["NUTS_ID"] = panel["NUTS_ID"].astype(str)

In [ ]:
# 用 general_per_million 生成 ln_x（自动把 ≤0 的设为 NaN，后面会 dropna）
panel["general_per_million"] = pd.to_numeric(panel["general_per_million"], errors="coerce")
bad = (panel["general_per_million"] <= 0).sum()
print(f"≤0 的条数：{bad}")  # 仅提示你是否存在 0/负数
panel["ln_x"] = np.log(panel["general_per_million"].where(panel["general_per_million"] > 0, np.nan))

In [ ]:
# 用已对数的 y（log_gdp_pc）作为 ln_y
panel["ln_y"] = pd.to_numeric(panel["log_gdp_pc"], errors="coerce")

In [ ]:
s = panel["general_per_million"].astype(str).str.strip()

# 1) 处理空白与常见“缺失占位”
s = (s.replace({"": None, "NA": None, "N/A": None, ".": None, "-": None, "—": None})
       .str.replace("\u00A0", "", regex=False))  # 去掉不换行空格

# 2) 先去掉千分位逗号（123,456 → 123456），再把小数逗号换成点（1,23 → 1.23）
s = s.str.replace(r",(?!\d{3}\b)", ".", regex=True)         # 小数逗号 → 点
s = s.str.replace(r"(?<=\d),(?=\d{3}\b)", "", regex=True)   # 千分位逗号 → 空

# 3) 去掉除数字/点/负号外的杂字符（比如 <、~、%）
s = s.str.replace(r"[^0-9\.\-]", "", regex=True)

# 4) 转成数值
panel["general_per_million"] = pd.to_numeric(s, errors="coerce")

# 5) 统计问题行
n_all  = len(s)
n_na   = panel["general_per_million"].isna().sum()
n_le0  = (panel["general_per_million"] <= 0).sum()
print(f"总行数: {n_all} | 解析失败(→NaN): {n_na} | ≤0 行: {n_le0}")

# 6) 生成 ln_x（把 ≤0 当缺失丢掉；若不想丢见下方“平移法”）
panel["ln_x"] = np.log(panel["general_per_million"].where(panel["general_per_million"] > 0, np.nan))

In [ ]:
key_id = "NUTS_ID" if "NUTS_ID" in panel.columns else "region"
panel[key_id] = panel[key_id].astype(str)
panel["year"] = pd.to_numeric(panel["year"], errors="coerce").astype("Int64")

# === 2) 排序并生成滞后 ===============================================
panel = panel.sort_values([key_id, "year"]).reset_index(drop=True)

def add_lags(df, id_col, cols, lags=(1,2,3)):
    for c in cols:
        for L in lags:
            df[f"{c}_lag{L}"] = df.groupby(id_col, observed=True)[c].shift(L)
    return df

panel = add_lags(panel, key_id, cols=["ln_y", "ln_x"], lags=(1,2,3))

# === 3) 快速查看（前几行） ===========================================
cols_show = [
    key_id, "year",
    "ln_y","ln_y_lag1","ln_y_lag2","ln_y_lag3",
    "ln_x","ln_x_lag1","ln_x_lag2","ln_x_lag3"
]
print(panel[cols_show].head(12))

In [ ]:
# Queen 邻接
wq = Queen.from_dataframe(nuts, silence_warnings=True)

# KNN6（补孤岛/断裂）
wk = KNN.from_dataframe(nuts, k=6)

# 合并 Queen 与 KNN 的邻接
neighbors = {}
for i in range(nuts.shape[0]):
    qn = wq.neighbors.get(i, [])
    kn = wk.neighbors.get(i, [])
    neighbors[i] = sorted(set(qn + kn))

W = weights.W(neighbors)     # 初始1/0权重
W.transform = "R"            # 行标准化（常用）
n = W.n

# 建立从地区ID到W行号的映射
nuts = nuts.reset_index(drop=True)
id2row = dict(zip(nuts["NUTS_ID"], nuts.index))
panel["rid"] = panel["NUTS_ID"].map(id2row)


In [ ]:
# 时间滞后 y_{i,t-1}、y_{i,t-2}、y_{i,t-3}
panel = panel.sort_values(["NUTS_ID","year"])
panel["ln_y_lag1"] = panel.groupby("NUTS_ID")["ln_y"].shift(1)
panel["ln_y_lag2"] = panel.groupby("NUTS_ID")["ln_y"].shift(2)
panel["ln_y_lag3"] = panel.groupby("NUTS_ID")["ln_y"].shift(3)

# 分年份做空间乘：Wy、Wx
def add_spatial_lag(df, colname, newname):
    out = []
    for t, g in df.groupby("year"):
        v = g.set_index("rid")[colname].reindex(range(n)).values
        # 缺失填充为 0（更稳妥做法是先 dropna 再对齐；这里简单演示）
        v = np.nan_to_num(v, nan=0.0)
        lagv = W.sparse @ v
        tmp = pd.DataFrame({"rid": range(n), "year": t, newname: lagv})
        out.append(tmp)
    out = pd.concat(out, ignore_index=True)
    return df.merge(out, on=["rid","year"], how="left")

panel = add_spatial_lag(panel, "ln_y", "wy")     # W ln Y_it  —— ρ 的右手项
panel = add_spatial_lag(panel, "ln_x", "wx")     # W ln X_it  —— θ 的右手项

# 作为工具的高阶 W * X、W^2 * X，以及 W * y_{t-2}
# W^2 * x
def add_higher_order_Wx(df, base_name="ln_x"):
    # W^2：直接连乘稀疏矩阵
    W2 = W.sparse @ W.sparse
    out = []
    for t, g in df.groupby("year"):
        x = g.set_index("rid")[base_name].reindex(range(n)).values
        x = np.nan_to_num(x, nan=0.0)
        w2x = W2 @ x
        tmp = pd.DataFrame({"rid": range(n), "year": t, "w2_ln_x": w2x})
        out.append(tmp)
    return df.merge(pd.concat(out, ignore_index=True), on=["rid","year"], how="left")

panel = add_higher_order_Wx(panel, "ln_x")

# W * y_{t-2}
panel["ln_y_lag2_tmp"] = panel.groupby("NUTS_ID")["ln_y"].shift(2)
panel = add_spatial_lag(panel, "ln_y_lag2_tmp", "w_ln_y_lag2")
panel.drop(columns=["ln_y_lag2_tmp"], inplace=True)

# 回归可用的数据（去掉首两期缺滞后值的行）
reg = panel.dropna(subset=["ln_y","ln_x","ln_y_lag1","wy","wx","ln_y_lag2","ln_y_lag3","w2_ln_x","w_ln_y_lag2"]).copy()

In [ ]:
# 把 year / NUTS_ID 设为分类（固定效应用虚拟变量吸收）


reg["year"]    = reg["year"].astype("category")
reg["NUTS_ID"] = reg["NUTS_ID"].astype("category")

# 公式： y ~ exog + [endog ~ instruments]
# 这里 exog 有 ln_x, wx, 以及双向 FE：C(year) + C(NUTS_ID)
# endog 是 ln_y_lag1（δ）和 wy（ρ）
# instruments（排除工具）包括：w2_ln_x, ln_y_lag2, ln_y_lag3, w_ln_y_lag2
formula = """
ln_y ~ 1 + C(year) + C(NUTS_ID) + ln_x + wx
      + [ ln_y_lag1 + wy ~ w2_ln_x + ln_y_lag2 + ln_y_lag3 + w_ln_y_lag2 ]
"""

iv = IV2SLS.from_formula(formula, data=reg)
res = iv.fit(cov_type="robust")   # 或 "clustered", clusters=reg["NUTS_ID"]
print(res.summary)

# 抽出四个系数
delta  = res.params["ln_y_lag1"]
rho    = res.params["wy"]
beta   = res.params["ln_x"]
theta  = res.params["wx"]
delta, rho, beta, theta

In [ ]:
from scipy.sparse import csr_matrix

I = identity(n, format="csr")
Ws = W.sparse  # 行标准化后的稀疏矩阵

# 稀疏逆（n≈~200 能承受；更大可以用求解-迹近似）
M = spinv(I - rho * Ws)              # (I - ρW)^{-1}
S0 = M @ (beta * I + theta * Ws)     # 当期乘数

# 直接/总/间接（平均意义下）
direct_short  = S0.diagonal().mean()
total_short   = np.asarray(S0.sum(axis=1)).ravel().mean()
indirect_short = total_short - direct_short

# 长期（稳态）
mult = 1.0 / (1.0 - delta)
direct_long   = mult * direct_short
indirect_long = mult * indirect_short
total_long    = mult * total_short

print("Short-run effects  (elasticities):")
print(f"  Direct  = {direct_short:.4f}")
print(f"  Indirect= {indirect_short:.4f}")
print(f"  Total   = {total_short:.4f}")

print("\nLong-run effects (× 1/(1-δ)):")
print(f"  Direct  = {direct_long:.4f}")
print(f"  Indirect= {indirect_long:.4f}")
print(f"  Total   = {total_long:.4f}")

In [ ]:
from scipy.stats import norm

def stars(p):
    return '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

def sdm_impacts_with_se(res, W, x_name='ln_x', wy_name='wy', wx_name='wx', lagy_name='ln_y_lag1',
                        B=3000, seed=2025):
    """返回 impacts（SR/LR）的点估计、SE、p值、95%CI 以及占比AIE/ATE（含区间）"""
    n = W.n
    I = identity(n, format='csr')
    S = W.sparse

    p = res.params
    has_delta = lagy_name in p.index
    # 读参数（容错：x_name 换成 'x' 也能取到）
    rho   = float(p[wy_name])
    beta  = float(p[x_name] if x_name in p.index else p['x'])
    theta = float(p[wx_name])
    delta = float(p[lagy_name]) if has_delta else 0.0

    # —— 点估计（SR/LR）
    M  = spinv(I - rho * S)
    S0 = M @ (beta * I + theta * S)
    direct_SR   = S0.diagonal().mean()
    total_SR    = np.asarray(S0.sum(axis=1)).ravel().mean()
    indirect_SR = total_SR - direct_SR
    mult        = 1.0 / (1.0 - delta) if has_delta else 1.0
    direct_LR, indirect_LR, total_LR = mult*direct_SR, mult*indirect_SR, mult*total_SR

    # —— 参数模拟（Krinsky–Robb）
    names = [wy_name, x_name if x_name in p.index else 'x', wx_name] + ([lagy_name] if has_delta else [])
    cov   = res.cov.loc[names, names].values
    mean  = p[names].values

    rng   = np.random.default_rng(seed)
    draws = rng.multivariate_normal(mean, cov, size=B)

    SR_dir=[]; SR_ind=[]; SR_tot=[]
    LR_dir=[]; LR_ind=[]; LR_tot=[]
    for d in draws:
        if has_delta:
            rho_b, beta_b, theta_b, delta_b = d
            mult_b = 1.0 / (1.0 - delta_b) if abs(delta_b) < 0.999 else np.nan
        else:
            rho_b, beta_b, theta_b = d
            mult_b = 1.0
        try:
            Mb = spinv(I - rho_b * S)
        except Exception:
            continue
        S0b = Mb @ (beta_b * I + theta_b * S)
        d_sr = S0b.diagonal().mean()
        t_sr = np.asarray(S0b.sum(axis=1)).ravel().mean()
        i_sr = t_sr - d_sr
        SR_dir.append(d_sr); SR_ind.append(i_sr); SR_tot.append(t_sr)
        LR_dir.append(mult_b*d_sr); LR_ind.append(mult_b*i_sr); LR_tot.append(mult_b*t_sr)

    def _summ(samples, est):
        arr = np.asarray(samples)
        se  = np.nanstd(arr, ddof=1)
        z   = est / se
        p   = 2 * (1 - norm.cdf(abs(z)))
        lo, hi = np.nanpercentile(arr, [2.5, 97.5])
        return dict(est=est, se=se, p=p, lo=lo, hi=hi)

    out = {
        ('Direct','SR'): _summ(SR_dir, direct_SR),
        ('Indirect','SR'): _summ(SR_ind, indirect_SR),
        ('Total','SR'): _summ(SR_tot, total_SR),
        ('Direct','LR'): _summ(LR_dir, direct_LR),
        ('Indirect','LR'): _summ(LR_ind, indirect_LR),
        ('Total','LR'): _summ(LR_tot, total_LR),
        ('Share','SR'):  _summ(np.asarray(SR_ind)/np.asarray(SR_tot), indirect_SR/total_SR),
        ('Share','LR'):  _summ(np.asarray(LR_ind)/np.asarray(LR_tot), indirect_LR/total_LR)
    }
    return out, has_delta
# --- helper for stars (if not already defined) ---
def stars(p):
    return '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

# === 1) Impacts ===
imp, has_delta = sdm_impacts_with_se(
    res, W, x_name='ln_x', wy_name='wy', wx_name='wx', lagy_name='ln_y_lag1'
)

# === 2) Panel A: coefficients ===
coef_rows = []
labels = {
    'wy'        : 'ρ · W ln Y',
    'ln_x'      : 'β · ln X',
    'x'         : 'β · ln X',
    'wx'        : 'θ · W ln X',
    'ln_y_lag1' : 'δ · ln Y_{t−1}'
}
for name in ['wy', 'ln_x' if 'ln_x' in res.params.index else 'x', 'wx'] + (['ln_y_lag1'] if has_delta else []):
    coef_rows.append({
        'Variable'   : labels[name],
        'Coef.'      : res.params[name],
        'Std. Error' : res.std_errors[name],
        'p-value'    : res.pvalues[name]
    })
panelA = pd.DataFrame(coef_rows)

# sample/setup info
key_id = 'NUTS_ID' if 'NUTS_ID' in panel.columns else 'region'
try:
    N = int(getattr(res, 'nobs', None) or reg.shape[0])
except NameError:
    N = int(getattr(res, 'nobs', np.nan))
G = int(reg[key_id].nunique())
T = int(reg['year'].nunique())

info_rows = pd.DataFrame([
    {'Variable':'Region FE / Year FE', 'Coef.':'Yes / Yes', 'Std. Error':'', 'p-value':''},
    {'Variable':'Obs. N; Regions G; Years T', 'Coef.':f'{N}; {G}; {T}', 'Std. Error':'', 'p-value':''}
])
panelA_full = pd.concat([panelA, info_rows], ignore_index=True)

# === 3) Panel B: impacts (short/long with SE & p) ===
def fmt(cell):
    return f"{cell['est']:.4f} ({cell['se']:.4f}){stars(cell['p'])}"

panelB = pd.DataFrame({
    'Effect'          : ['Direct (ADE)','Indirect (AIE)','Total (ATE)','Spillover share (AIE/ATE)'],
    'Short-run (SR)'  : [fmt(imp[('Direct','SR')]),
                         fmt(imp[('Indirect','SR')]),
                         fmt(imp[('Total','SR')]),
                         f"{imp[('Share','SR')]['est']:.3f} [{imp[('Share','SR')]['lo']:.3f},{imp[('Share','SR')]['hi']:.3f}]"],
    'Long-run (LR)'   : [fmt(imp[('Direct','LR')]),
                         fmt(imp[('Indirect','LR')]),
                         fmt(imp[('Total','LR')]),
                         f"{imp[('Share','LR')]['est']:.3f} [{imp[('Share','LR')]['lo']:.3f},{imp[('Share','LR')]['hi']:.3f}]"]
})

# === 4) Combine and export ===
empty = pd.DataFrame([{'Variable':'', 'Coef.':'', 'Std. Error':'', 'p-value':''}])

combined = pd.concat([
    pd.DataFrame([{'Variable':'Panel A: Coefficients (dependent variable ln Y)', 'Coef.':'', 'Std. Error':'', 'p-value':''}]),
    panelA_full,
    empty,
    pd.DataFrame([{'Variable':'Panel B: LeSage–Pace impacts (ln X → ln Y, elasticities)', 'Coef.':'', 'Std. Error':'', 'p-value':''}]),
    panelB.rename(columns={'Effect':'Variable', 'Short-run (SR)':'Coef.', 'Long-run (LR)':'Std. Error'}).assign(**{'p-value':''})
], ignore_index=True)

# CSV
combined.to_csv(OUT_DIR / "SDM_general_gdp2.csv", index=False, encoding="utf-8-sig")

# Excel (engine fallback: xlsxwriter -> openpyxl; if neither, only CSV)
excel_path = OUT_DIR / "SDM_general_gdp2.xlsx"
engine = None
try:
    import xlsxwriter  # noqa
    engine = "xlsxwriter"
except ModuleNotFoundError:
    try:
        import openpyxl  # noqa
        engine = "openpyxl"
    except ModuleNotFoundError:
        engine = None

if engine:
    with pd.ExcelWriter(excel_path, engine=engine) as w:
        combined.to_excel(w, index=False, sheet_name="SDM")
    print(f"Exported Excel (engine={engine}): {excel_path}")
else:
    print("xlsxwriter/openpyxl not installed; exported CSV only. Install one if you need .xlsx.")

In [ ]:
# 只做一次
# pip install geopandas libpysal linearmodels scipy pandas numpy
from pathlib import Path   # ← 关键：先导入 Path
import numpy as np
import pandas as pd
import geopandas as gpd

from libpysal import weights
from libpysal.weights import Queen, KNN
from scipy.sparse import identity
from scipy.sparse.linalg import inv as spinv

from linearmodels.iv import IV2SLS   # 用 2SLS/IV 估计 δ、ρ、β、θ（带双向固定效应）

In [ ]:
PATH_PANEL = Path(r"D:\Dissertation\dissertation\data 2\data\Without UK and Germany\Final\panel_long_merged_with_general.csv")
PATH_GEO   = Path(r"D:\Dissertation\dissertation\data 2\data\Without UK and Germany\Final\cleaned geo data\NUTS2_2021.geojson")
OUT_DIR    = Path(r"D:\Dissertation\dissertation\data 2\data\Without UK and Germany\Final\moran\SDM_Edit")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ── 读数据 ─────────────────────────────────────────────────────────────
panel = pd.read_csv(PATH_PANEL)                       # 长表：NUTS_ID, year, ...
nuts  = gpd.read_file(PATH_GEO)[["NUTS_ID","geometry"]]


panel = panel.rename(columns={"region": "NUTS_ID"})   # 关键一行
panel["NUTS_ID"] = panel["NUTS_ID"].astype(str)

# 用 general_per_million 生成 ln_x（自动把 ≤0 的设为 NaN，后面会 dropna）
panel["general_per_million"] = pd.to_numeric(panel["general_per_million"], errors="coerce")
bad = (panel["vet_per_million"] <= 0).sum()
print(f"≤0 的条数：{bad}")  # 仅提示你是否存在 0/负数
panel["ln_x"] = np.log(panel["general_per_million"].where(panel["general_per_million"] > 0, np.nan))


In [ ]:
# 用已对数的 y（log_gdp_pc）作为 ln_y
panel["employment_rate"] = pd.to_numeric(panel["employment_rate"], errors="coerce")

er = panel["employment_rate"]
# 自动识别：若最大值>1，说明是百分数，先 /100
er = er/100.0 if er.dropna().max() > 1.0000001 else er

print("employment_rate 中 ≤0 的条数：", (er <= 0).sum())
panel["ln_y"] = np.log(er.where(er > 0, np.nan))   # ln(employment_rate)

s = panel["general_per_million"].astype(str).str.strip()

# 1) 处理空白与常见“缺失占位”
s = (s.replace({"": None, "NA": None, "N/A": None, ".": None, "-": None, "—": None})
       .str.replace("\u00A0", "", regex=False))  # 去掉不换行空格

# 2) 先去掉千分位逗号（123,456 → 123456），再把小数逗号换成点（1,23 → 1.23）
s = s.str.replace(r",(?!\d{3}\b)", ".", regex=True)         # 小数逗号 → 点
s = s.str.replace(r"(?<=\d),(?=\d{3}\b)", "", regex=True)   # 千分位逗号 → 空

# 3) 去掉除数字/点/负号外的杂字符（比如 <、~、%）
s = s.str.replace(r"[^0-9\.\-]", "", regex=True)

# 4) 转成数值
panel["general_per_million"] = pd.to_numeric(s, errors="coerce")

# 5) 统计问题行
n_all  = len(s)
n_na   = panel["general_per_million"].isna().sum()
n_le0  = (panel["general_per_million"] <= 0).sum()
print(f"总行数: {n_all} | 解析失败(→NaN): {n_na} | ≤0 行: {n_le0}")

# 6) 生成 ln_x（把 ≤0 当缺失丢掉；若不想丢见下方“平移法”）
panel["ln_x"] = np.log(panel["general_per_million"].where(panel["general_per_million"] > 0, np.nan))
                                                            

In [ ]:
# 用 general_per_million 生成 ln_x（自动把 ≤0 的设为 NaN，后面会 dropna）
panel["general_per_million"] = pd.to_numeric(panel["general_per_million"], errors="coerce")
bad = (panel["general_per_million"] <= 0).sum()
print(f"≤0 的条数：{bad}")  # 仅提示你是否存在 0/负数
panel["ln_x"] = np.log(panel["general_per_million"].where(panel["general_per_million"] > 0, np.nan))

In [ ]:
key_id = "NUTS_ID" if "NUTS_ID" in panel.columns else "region"
panel[key_id] = panel[key_id].astype(str)
panel["year"] = pd.to_numeric(panel["year"], errors="coerce").astype("Int64")

# === 2) 排序并生成滞后 ===============================================
panel = panel.sort_values([key_id, "year"]).reset_index(drop=True)

def add_lags(df, id_col, cols, lags=(1,2,3)):
    for c in cols:
        for L in lags:
            df[f"{c}_lag{L}"] = df.groupby(id_col, observed=True)[c].shift(L)
    return df

panel = add_lags(panel, key_id, cols=["ln_y", "ln_x"], lags=(1,2,3))

# === 3) 快速查看（前几行） ===========================================
cols_show = [
    key_id, "year",
    "ln_y","ln_y_lag1","ln_y_lag2","ln_y_lag3",
    "ln_x","ln_x_lag1","ln_x_lag2","ln_x_lag3"
]
print(panel[cols_show].head(12))

In [ ]:
# Queen 邻接
wq = Queen.from_dataframe(nuts, silence_warnings=True)

# KNN6（补孤岛/断裂）
wk = KNN.from_dataframe(nuts, k=6)

# 合并 Queen 与 KNN 的邻接
neighbors = {}
for i in range(nuts.shape[0]):
    qn = wq.neighbors.get(i, [])
    kn = wk.neighbors.get(i, [])
    neighbors[i] = sorted(set(qn + kn))

W = weights.W(neighbors)     # 初始1/0权重
W.transform = "R"            # 行标准化（常用）
n = W.n

# 建立从地区ID到W行号的映射
nuts = nuts.reset_index(drop=True)
id2row = dict(zip(nuts["NUTS_ID"], nuts.index))
panel["rid"] = panel["NUTS_ID"].map(id2row)

In [ ]:
# 时间滞后 y_{i,t-1}、y_{i,t-2}、y_{i,t-3}
panel = panel.sort_values(["NUTS_ID","year"])
panel["ln_y_lag1"] = panel.groupby("NUTS_ID")["ln_y"].shift(1)
panel["ln_y_lag2"] = panel.groupby("NUTS_ID")["ln_y"].shift(2)
panel["ln_y_lag3"] = panel.groupby("NUTS_ID")["ln_y"].shift(3)

# 分年份做空间乘：Wy、Wx
def add_spatial_lag(df, colname, newname):
    out = []
    for t, g in df.groupby("year"):
        v = g.set_index("rid")[colname].reindex(range(n)).values
        # 缺失填充为 0（更稳妥做法是先 dropna 再对齐；这里简单演示）
        v = np.nan_to_num(v, nan=0.0)
        lagv = W.sparse @ v
        tmp = pd.DataFrame({"rid": range(n), "year": t, newname: lagv})
        out.append(tmp)
    out = pd.concat(out, ignore_index=True)
    return df.merge(out, on=["rid","year"], how="left")

panel = add_spatial_lag(panel, "ln_y", "wy")     # W ln Y_it  —— ρ 的右手项
panel = add_spatial_lag(panel, "ln_x", "wx")     # W ln X_it  —— θ 的右手项

# 作为工具的高阶 W * X、W^2 * X，以及 W * y_{t-2}
# W^2 * x
def add_higher_order_Wx(df, base_name="ln_x"):
    # W^2：直接连乘稀疏矩阵
    W2 = W.sparse @ W.sparse
    out = []
    for t, g in df.groupby("year"):
        x = g.set_index("rid")[base_name].reindex(range(n)).values
        x = np.nan_to_num(x, nan=0.0)
        w2x = W2 @ x
        tmp = pd.DataFrame({"rid": range(n), "year": t, "w2_ln_x": w2x})
        out.append(tmp)
    return df.merge(pd.concat(out, ignore_index=True), on=["rid","year"], how="left")

panel = add_higher_order_Wx(panel, "ln_x")

# W * y_{t-2}
panel["ln_y_lag2_tmp"] = panel.groupby("NUTS_ID")["ln_y"].shift(2)
panel = add_spatial_lag(panel, "ln_y_lag2_tmp", "w_ln_y_lag2")
panel.drop(columns=["ln_y_lag2_tmp"], inplace=True)

# 回归可用的数据（去掉首两期缺滞后值的行）
reg = panel.dropna(subset=["ln_y","ln_x","ln_y_lag1","wy","wx","ln_y_lag2","ln_y_lag3","w2_ln_x","w_ln_y_lag2"]).copy()

In [ ]:
# 把 year / NUTS_ID 设为分类（固定效应用虚拟变量吸收）


reg["year"]    = reg["year"].astype("category")
reg["NUTS_ID"] = reg["NUTS_ID"].astype("category")

# 公式： y ~ exog + [endog ~ instruments]
# 这里 exog 有 ln_x, wx, 以及双向 FE：C(year) + C(NUTS_ID)
# endog 是 ln_y_lag1（δ）和 wy（ρ）
# instruments（排除工具）包括：w2_ln_x, ln_y_lag2, ln_y_lag3, w_ln_y_lag2
formula = """
ln_y ~ 1 + C(year) + C(NUTS_ID) + ln_x + wx
      + [ ln_y_lag1 + wy ~ w2_ln_x + ln_y_lag2 + ln_y_lag3 + w_ln_y_lag2 ]
"""

iv = IV2SLS.from_formula(formula, data=reg)
res = iv.fit(cov_type="robust")   # 或 "clustered", clusters=reg["NUTS_ID"]
print(res.summary)

# 抽出四个系数
rho    = res.params["wy"]
beta   = res.params["ln_x"]
theta  = res.params["wx"]
delta, rho, beta, theta

In [ ]:
from scipy.sparse import csr_matrix

I = identity(n, format="csr")
Ws = W.sparse  # 行标准化后的稀疏矩阵

# 稀疏逆（n≈~200 能承受；更大可以用求解-迹近似）
M = spinv(I - rho * Ws)              # (I - ρW)^{-1}
S0 = M @ (beta * I + theta * Ws)     # 当期乘数

# 直接/总/间接（平均意义下）
direct_short  = S0.diagonal().mean()
total_short   = np.asarray(S0.sum(axis=1)).ravel().mean()
indirect_short = total_short - direct_short

# 长期（稳态）
mult = 1.0 / (1.0 - delta)
direct_long   = mult * direct_short
indirect_long = mult * indirect_short
total_long    = mult * total_short

print("Short-run effects  (elasticities):")
print(f"  Direct  = {direct_short:.4f}")
print(f"  Indirect= {indirect_short:.4f}")
print(f"  Total   = {total_short:.4f}")

print("\nLong-run effects (× 1/(1-δ)):")
print(f"  Direct  = {direct_long:.4f}")
print(f"  Indirect= {indirect_long:.4f}")
print(f"  Total   = {total_long:.4f}")


In [ ]:
# --- helper for stars (if not already defined) ---
def stars(p):
    return '***' if p < 0.01 else '**' if p < 0.05 else '*' if p < 0.10 else ''

# === 1) Impacts ===
imp, has_delta = sdm_impacts_with_se(
    res, W, x_name='ln_x', wy_name='wy', wx_name='wx', lagy_name='ln_y_lag1'
)

# === 2) Panel A: coefficients ===
coef_rows = []
labels = {
    'wy'        : 'ρ · W ln Y',
    'ln_x'      : 'β · ln X',
    'x'         : 'β · ln X',
    'wx'        : 'θ · W ln X',
    'ln_y_lag1' : 'δ · ln Y_{t−1}'
}
for name in ['wy', 'ln_x' if 'ln_x' in res.params.index else 'x', 'wx'] + (['ln_y_lag1'] if has_delta else []):
    coef_rows.append({
        'Variable'   : labels[name],
        'Coef.'      : res.params[name],
        'Std. Error' : res.std_errors[name],
        'p-value'    : res.pvalues[name]
    })
panelA = pd.DataFrame(coef_rows)

# sample/setup info
key_id = 'NUTS_ID' if 'NUTS_ID' in panel.columns else 'region'
try:
    N = int(getattr(res, 'nobs', None) or reg.shape[0])
except NameError:
    N = int(getattr(res, 'nobs', np.nan))
G = int(reg[key_id].nunique())
T = int(reg['year'].nunique())

info_rows = pd.DataFrame([
    {'Variable':'Region FE / Year FE', 'Coef.':'Yes / Yes', 'Std. Error':'', 'p-value':''},
    {'Variable':'Obs. N; Regions G; Years T', 'Coef.':f'{N}; {G}; {T}', 'Std. Error':'', 'p-value':''}
])
panelA_full = pd.concat([panelA, info_rows], ignore_index=True)

# === 3) Panel B: impacts (short/long with SE & p) ===
def fmt(cell):
    return f"{cell['est']:.4f} ({cell['se']:.4f}){stars(cell['p'])}"

panelB = pd.DataFrame({
    'Effect'          : ['Direct (ADE)','Indirect (AIE)','Total (ATE)','Spillover share (AIE/ATE)'],
    'Short-run (SR)'  : [fmt(imp[('Direct','SR')]),
                         fmt(imp[('Indirect','SR')]),
                         fmt(imp[('Total','SR')]),
                         f"{imp[('Share','SR')]['est']:.3f} [{imp[('Share','SR')]['lo']:.3f},{imp[('Share','SR')]['hi']:.3f}]"],
    'Long-run (LR)'   : [fmt(imp[('Direct','LR')]),
                         fmt(imp[('Indirect','LR')]),
                         fmt(imp[('Total','LR')]),
                         f"{imp[('Share','LR')]['est']:.3f} [{imp[('Share','LR')]['lo']:.3f},{imp[('Share','LR')]['hi']:.3f}]"]
})

# === 4) Combine and export ===
empty = pd.DataFrame([{'Variable':'', 'Coef.':'', 'Std. Error':'', 'p-value':''}])

combined = pd.concat([
    pd.DataFrame([{'Variable':'Panel A: Coefficients (dependent variable ln Y)', 'Coef.':'', 'Std. Error':'', 'p-value':''}]),
    panelA_full,
    empty,
    pd.DataFrame([{'Variable':'Panel B: LeSage–Pace impacts (ln X → ln Y, elasticities)', 'Coef.':'', 'Std. Error':'', 'p-value':''}]),
    panelB.rename(columns={'Effect':'Variable', 'Short-run (SR)':'Coef.', 'Long-run (LR)':'Std. Error'}).assign(**{'p-value':''})
], ignore_index=True)

# CSV
combined.to_csv(OUT_DIR / "SDM_general_vet_em3.csv", index=False, encoding="utf-8-sig")

# Excel (engine fallback: xlsxwriter -> openpyxl; if neither, only CSV)
excel_path = OUT_DIR / "SDM_general_vet_em3.xlsx"
engine = None
try:
    import xlsxwriter  # noqa
    engine = "xlsxwriter"
except ModuleNotFoundError:
    try:
        import openpyxl  # noqa
        engine = "openpyxl"
    except ModuleNotFoundError:
        engine = None

if engine:
    with pd.ExcelWriter(excel_path, engine=engine) as w:
        combined.to_excel(w, index=False, sheet_name="SDM")
    print(f"Exported Excel (engine={engine}): {excel_path}")
else:
    print("xlsxwriter/openpyxl not installed; exported CSV only. Install one if you need .xlsx.")
